Ayudantía Martes 28 de agosto de 2019

# Actividad Web Scraping

### Nombre: Benjamín Farías V.

## Indicaciones generales :

### • En esta actividad deberan ocupar Web Scraping para obtener una base de datos a partir de los datos de bolsos en https://www.newchic.com/es/.
### •  Deben obtener los datos de 100 productos con los atributos de: 
    •  Nombre
    •  Rating
    •  Precio
    •  Oferta
    •  Puntos
    •  Color
    •  Envío
    •  Especificación
    •  Comentarios (obtener primeros 5)

### •  Para esta actividad se debe entregar Notebook y archivo CSV con la base de datos.
### • Entrega martes 27 de agosto a las 10:59 pm.


## Elementos a usar para desarrollar esta actividad :

• Usar find(), find all() y select(). Todos permiten la selección de tags. A continuación se muestra un ejemplo de estructura html:
    - Tipo de tag
        Ej. <a>, <div ...>
    - Valores de atributos
        Ej. <div class=”abclinksmain”>)
    - N-ésimo hijo de/ Hermano de... <div class=”abclinksmain”>
        <ul>
        <li>—–Primer hijo de <ul> ...
        </li>
        <li>—–Segundo hijo de <ul>
        <a href=”http://www. ...”>
        A </a>
        </li>
        <li>—–Tercer hijo de <ul> ...
        </li>
        </u> </div>
    
• Las funciones find() y find all() reciben como parámetro filtros para seleccionar tags. Por otra parte, select() acepta selectores CSS para el filtro. El modo de uso de ambos se indica en la documentación (*).
    - Detalles de BeautifulSoup:
         Para extraer el valor de un atributo de un tag deben usar tag[“atributo”].
         Para acceder a su texto interior deben usar tag.text.
    
• Obtener página de Inicio : Lo primero que deben hacer es obtener la data html de la página principal haciendo uso de la librería requests. Posteriormente, deben procesar su data a un objeto BeautifulSoup para poder navegar y buscar fácilmente a través del árbol.
       - result = requests.get(path)
       - content = result.content
       - soup = BeautifulSoup(content, 'html.parser')

• Obtener url directorios : Si bien en el lapso de tiempo de esta clase no se descargará toda la base de datos, es necesario poder hacer una navegación profunda del sitio.
    
• Manejo de datos : Poner atención en el manejo de errores en casos en que algún atributo no exista. De no existir, rellenar con string o lista vacía ese atributo en la base de datos.
        
### (*) Para entender detalles de el uso de BeautifulSoup consultar la documentación https://www.crummy.com/software/BeautifulSoup/bs4/doc/


## 1. Librerías

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

## 2. Funciones para extraer información de los productos con los atributos que se señalan.

In [23]:
# Función que obtiene toda la información de los productos de la página
def get_products_info(url, results_list):
    content = requests.get(url)
    content.encoding = 'utf-8'
    soup = BeautifulSoup(content.text, "lxml")
    data = soup.find("ul", {"class": "clearfixes newlst_box product-list-js"}).find_all(
        "li", {"class": "wom_lst_detail product-item-js"})

    # Se itera sobre cada uno de ellos, obteniendo sus atributos en un diccionario
    for bag in data:
        product_url = bag.find("div", {"class": "wom_lst_pic product-image-js"}).find("a", href=True)['href']
        current_request = requests.get(product_url)
        current_request.encoding = 'utf-8'
        current = BeautifulSoup(current_request.text, "lxml")
        product = {}

        # Nombre
        name = current.find("h1", {"class": "d-inline product-info-product-name-js"}).text
        product["Name"] = name

        # Rating
        rating = current.find("div", {"class": "flex align-items-center mx-lg-n2 mt-lg-5"}).find("span").text
        product["Rating"] = float(rating[:3])

        # Precio
        price = current.find("span", {"class": "px-lg-2 price-number"})['data-sale-price']
        product["Price"] = float(price)

        # Oferta
        discount = current.find(
            "span", {"class": "px-lg-5 bg-primary text-white font-small product-price__discount product-price-discount-js"})[
            'data-discount']
        product["Discount (%)"] = int(discount)

        # Puntos (no se pueden obtener, ya que son dinámicos)
        points = current.find("li", {"class": "flex mx-lg-n2"})
        product["Points"] = points.find("div", {"class": "lg px-lg-2"}).text.strip()

        # Colores
        images = current.find_all(
            "div", {"class": "product-info__attr-image product-info-attr-image-js product-info-attr-image-ga-js"})
        colors = [x.find("img")['title'] for x in images]
        if not colors:
            colors = ['No Disponible!']
        product["Colors"] = ",".join(colors)

        # Envío
        shipping = current.find("div", {"class": "product-info-ship-msg-js"}).text
        product["Shipping"] = shipping

        # Especificaciones
        description = current.find("div", {"class": "product-description-container"})
        specs = description.find("span").text
        product["Specs"] = specs

        # Comentarios
        comments = current.find("div", {"class": "reviews-list mb-lg-50"})
        if not comments:
            comments = ['No hay comentarios!']
        else:
            comments = [x.text.strip() for x in comments.find_all("div", {"class": "mb-lg-20"}) if x.text.strip()]
        product["Comments"] = ",".join(comments)

        # Se añaden a la lista
        results_list.append(product)
    return

## 3. Se extrae la información de varias páginas para obtener una mayor cantidad de productos.

In [24]:
# Se realiza la request a la página de los bolsos, separando la url para poder iterar sobre las distintas páginas 
# (en este caso son las primeras 3 páginas)

url_base = "https://es.newchic.com/bags-c-3579/"
url_extra = ".html?act=0&is_new=0&pagesize=35&sort=1&from=nav"

results = list()
for page in range(1, 4):
    new_url = f'{url_base}{page}{url_extra}'
    get_products_info(new_url, results)
results

[{'Name': 'Mujer Laptop de viaje de diseño Bolsa Crossbody sólido Bolsa  ',
  'Rating': 5.0,
  'Price': 36.99,
  'Discount (%)': 75,
  'Points': '',
  'Colors': 'Rosado,Azul,Negro,Marrón,Burdeos',
  'Shipping': 'en stock, enviado en Aug 28 - Aug 30',
  'Specs': 'especificación :Color: Negro,Azul,marrón,Rosado,Vino rojoEstilo: ViajarMaterial: PUTipo de patrón: Sólido',
  'Comments': 'i love it,Fantastic quality ! Even fits my 13” laptop for work ! ❤️❤️❤️❤️❤️,Сумка очень понравилась! Вместительная, удобная. Качество отличное! Упаковано хорошо. Посылка пришла достаточно быстро. Спасибо продавцу!,muito linda, melhor do que eu esperava, e é bem grande.,I liked it very much Amazing bag with larger space'},
 {'Name': 'QUEENIE Mujer Botella desmontable multifunción sólida Bolsa Hombro de compras Bolsa',
  'Rating': 5.0,
  'Price': 39.99,
  'Discount (%)': 56,
  'Points': '',
  'Colors': 'Azul,Marrón,Burdeos,Negro,Rosado',
  'Shipping': 'en stock, enviado en Aug 28 - Aug 30',
  'Specs': 'especi

## 4. Cargar base de datos en un dataframe y almacenar en un archivo con extensión CSV.

In [34]:
# Se cargan los datos a un DataFrame
data = pd.DataFrame(results)
data

,Name,Rating,Price,Discount (%),Points,Colors,Shipping,Specs,Comments
0,Mujer Laptop de viaje de diseño Bolsa Crossbod...,5.0,36.99,75,,"Rosado,Azul,Negro,Marrón,Burdeos","en stock, enviado en Aug 28 - Aug 30","especificación :Color: Negro,Azul,marrón,Rosad...","i love it,Fantastic quality ! Even fits my 13”..."
1,QUEENIE Mujer Botella desmontable multifunción...,5.0,39.99,56,,"Azul,Marrón,Burdeos,Negro,Rosado","en stock, enviado en Aug 28 - Aug 30","especificación :Color: Negro,Azul,marrón,Rosad...",The order came without damages and was good pa...
2,BRENICE Mochila vintage piel bovina,4.9,39.98,52,,"Café,Azul,Marrón","en stock, enviado en Aug 28 - Aug 30",Mochila monocolor disponible en varios colores...,"A bolsa é linda, do tamanho da medida da foto ..."
3,Mujer Tarjeta de cuero sintético sólido Bolsa ...,5.0,19.99,60,,"Burdeos,Azul,Rosa,Amarillo,Gris,Naranja,Negro",solo queda 5 - enviado en Aug 28 - Aug 30,"especificación :Color: Negro,Azul,Gris,Amarill...","No es tan expandible, es bastante rigida pero ..."
4,Cartera de animal a lunares con cuerda,5.0,5.99,74,,"Negro,Rojo,Celeste,Gris,Violeta,Rosado,Verde","en stock, enviado en Aug 28 - Aug 30",Material: PU,"Took longer than expected to get here, but it'..."
5,Mujer National Bucket Bolsa Crossbody de impre...,4.8,11.66,58,,"# 01,# 02","en stock, enviado en Aug 28 - Aug 30","especificación :Color: Negro,BeigeEstilo: Naci...",This bag is absolutely beautiful. Everywhere I...
6,QUEENIE Mujer Bolso en relieve informal Compra...,5.0,39.99,58,,"Black,Wine Red,Brown,Grey,Blue","esperado enviado en February 07,2106","especificación :Color: Negro,Azul,rojo,marrón,...",Arrived a day early...very good quality. In al...
7,Mujer Diseño Crossbody Multifunción Bolso Sóli...,4.9,34.99,70,,"Azul,Marrón,Rojo,Negro,Gris","en stock, enviado en Aug 28 - Aug 30","especificación :Color: Negro,Azul,rojo,marrón,...",Great product. Look 95% like in the photos. No...
8,Mujer PU de cuero Embrague Bolsa Tarjeta Bolsa...,4.8,25.99,53,,"Naranja,De color verde oscuro,Violeta,Azul,Gri...","en stock, enviado en Aug 28 - Aug 30","especificación :Color: Negro,Azul,rojo,marrón,...",love the many mini compartments. makes it conv...
9,Xiaomi Lecturer Backpack Bolsa,0.0,26.99,72,,"Black,Blue,White",esperado enviado en Sep 03 - Sep 13,"especificación :Color: Negro,Blanco,AzulEstilo...",No hay comentarios!


In [35]:
# Se guarda la información obtenida en un archivo .csv
with open("newchicBolsos.csv", "w", encoding="utf-8") as db:
    data.to_csv(db, sep=';', index=False, encoding='utf-8')

In [36]:
# Ahora podemos utilizarlo para analizar la información!
df = pd.read_csv("newchicBolsos.csv",  sep=';')
df

,Name,Rating,Price,Discount (%),Points,Colors,Shipping,Specs,Comments
0,Mujer Laptop de viaje de diseño Bolsa Crossbod...,5.0,36.99,75,NaN,"Rosado,Azul,Negro,Marrón,Burdeos","en stock, enviado en Aug 28 - Aug 30","especificación :Color: Negro,Azul,marrón,Rosad...","i love it,Fantastic quality ! Even fits my 13”..."
1,QUEENIE Mujer Botella desmontable multifunción...,5.0,39.99,56,NaN,"Azul,Marrón,Burdeos,Negro,Rosado","en stock, enviado en Aug 28 - Aug 30","especificación :Color: Negro,Azul,marrón,Rosad...",The order came without damages and was good pa...
2,BRENICE Mochila vintage piel bovina,4.9,39.98,52,NaN,"Café,Azul,Marrón","en stock, enviado en Aug 28 - Aug 30",Mochila monocolor disponible en varios colores...,"A bolsa é linda, do tamanho da medida da foto ..."
3,Mujer Tarjeta de cuero sintético sólido Bolsa ...,5.0,19.99,60,NaN,"Burdeos,Azul,Rosa,Amarillo,Gris,Naranja,Negro",solo queda 5 - enviado en Aug 28 - Aug 30,"especificación :Color: Negro,Azul,Gris,Amarill...","No es tan expandible, es bastante rigida pero ..."
4,Cartera de animal a lunares con cuerda,5.0,5.99,74,NaN,"Negro,Rojo,Celeste,Gris,Violeta,Rosado,Verde","en stock, enviado en Aug 28 - Aug 30",Material: PU,"Took longer than expected to get here, but it'..."
5,Mujer National Bucket Bolsa Crossbody de impre...,4.8,11.66,58,NaN,"# 01,# 02","en stock, enviado en Aug 28 - Aug 30","especificación :Color: Negro,BeigeEstilo: Naci...",This bag is absolutely beautiful. Everywhere I...
6,QUEENIE Mujer Bolso en relieve informal Compra...,5.0,39.99,58,NaN,"Black,Wine Red,Brown,Grey,Blue","esperado enviado en February 07,2106","especificación :Color: Negro,Azul,rojo,marrón,...",Arrived a day early...very good quality. In al...
7,Mujer Diseño Crossbody Multifunción Bolso Sóli...,4.9,34.99,70,NaN,"Azul,Marrón,Rojo,Negro,Gris","en stock, enviado en Aug 28 - Aug 30","especificación :Color: Negro,Azul,rojo,marrón,...",Great product. Look 95% like in the photos. No...
8,Mujer PU de cuero Embrague Bolsa Tarjeta Bolsa...,4.8,25.99,53,NaN,"Naranja,De color verde oscuro,Violeta,Azul,Gri...","en stock, enviado en Aug 28 - Aug 30","especificación :Color: Negro,Azul,rojo,marrón,...",love the many mini compartments. makes it conv...
9,Xiaomi Lecturer Backpack Bolsa,0.0,26.99,72,NaN,"Black,Blue,White",esperado enviado en Sep 03 - Sep 13,"especificación :Color: Negro,Blanco,AzulEstilo...",No hay comentarios!
